# **Instalar Biblioteca do TMDB**

In [ ]:
!pip install tmdbv3api
!pip install imdbpy

# **Importar Bibliotecas**

In [2]:
import tmdbv3api as tmdb
from imdb import IMDb
import pandas as pd
import csv
import json

# **Configuração da API do TMDB**

In [3]:
# instanciar a configuração da api
api = tmdb.TMDb()

# Ler a key para a api a partir de um arquivo
# Subir um json com a sua key no colab
with open('key.json', 'r') as f:
  key = f.read()

key = json.loads(key)
key = key['key']

# Pode subistituir key pela string da key diretamente também
api.api_key = key
# Configura o idioma de resposta da api
api.language = 'pt-BR'

# **Funções**

## **Obter dos Filmes no TMDB**

In [4]:
# Retorna um dicionário com todos os detalhes que a api do TMDB fornece
def get_movie_details(movie):
  movie_tools = tmdb.Movie()
  id = movie['id']
  return movie_tools.details(id)

# Retorna uma lista com os filmes em uma página do TMDB
# Cada página costuma ter 20 filmes
def get_movies_in_page(page):
  discover = tmdb.Discover()
  movie_list = discover.discover_movies({
      'primary_release_date.gte': '2000-01-01',
      'primary_release_date.lte': '2021-12-31',
      'sort_by': 'revenue.desc',
      'page': page
  })
  return list(map(get_movie_details, movie_list))

# Ober uma lista de filmes extraidos de quantidade de páginas (num_pages)
# Cada Filme é um dicionario em que os campos são as informações do filme
# Podemos passar a página inicial de busca e o passo para as outras páginas
# Cada página têm 20 filmes, o número de filmes obtidos no total é 20*num_pages
def get_movies(num_pages, start_page=1, step=1):
  movies = []
  for i in range(num_pages):
    movies += get_movies_in_page(start_page + i*step)
  return movies

## **Gerar arquivo CSV**

In [5]:
# Construir um dicionário contendo só as informações desejadas
# do filme (fieldnames)
def build_movie_struct(movie, fieldnames, moviefields):
  movie_struct = dict()
  for i in range(len(fieldnames)):
    movie_struct[fieldnames[i]] = movie[moviefields[i]]
  return movie_struct

# Cria um arquivo csv com os filme obtidos do TMDB
# No csv adicionamos alguns dos campos fornecidos pelo TMDB (mfields)
# dependendo dos campos que queremos (fieldnames)
def build_movie_list_csv(movies:list, fieldnames:list, mfields:list):
  with open('filme.csv', 'w', newline='') as movies_csv:
    writer = csv.DictWriter(movies_csv, fieldnames=fieldnames)
    writer.writeheader()
    for movie in movies:
      ms = build_movie_struct(movie, fieldnames, mfields)
      writer.writerow(ms)


# **Ponto de Entrada**

In [ ]:
# obter a lista de filmes (100 no total)
movies = get_movies(5)

# Informações Necessárias no CSV
atributos = ["id_TMDB", "id_IMDB", "titulo", "titulo_original", "sinopse", "duracao", "ano", 
             "classificacao", "situacao", "nacionalidade", "idioma_original", 
             "orcamento", "receita", "num_oscars"]

# Campos com as informações no objeto do filme do TMDB
atributos_tmdb = ["id", "imdb_id", "title", "original_title", "overview", "runtime",
                  "release_date", "adult", "status", "original_language",
                  "original_language", "budget", "revenue", "popularity"]

# Criar o arquivo csv
build_movie_list_csv(movies, atributos, atributos_tmdb)


In [ ]:
def get_imdb_movie_data(imdb, movie_id):
  return imdb.get_movie(movie_id)

def get_certification(movie):
  certificates = movie['certificates']
  for i in certificates:
    if i.find('United States') != -1:
      return i
  return 'desconhecida'

def insert_movie_details(imdb,movie_table):
  for index, row in movie_table.iterrows():
    movie_id = int(row['id_IMDB'][2:])
    movie = get_imdb_movie_data(imdb, movie_id)
    country = movie['countries'][0]
    year = movie['year']
    certification = get_certification(movie)
    movie_table.loc[index,'nacionalidade'] = country
    movie_table.loc[index,'ano'] = year
    movie_table.loc[index,'classificacao'] = certification


In [ ]:
# Inserir Classificação, Ano e Nacionalidade dos filmes usando o IMDB
imdb = IMDb()
insert_movie_details(imdb, movies_table)
movies_table.to_csv('filme_atualizado.csv', encoding='utf-8', index=False)

In [12]:
def get_num_oscars(imdb, movie_id):
  movie = imdb.get_movie(movie_id, info=['awards'])
  if 'awards' not in movie.keys():
    return 0

  n_oscars = 0

  for award in movie['awards']:
    if award['award'].find('Oscar') != -1:
      if award['result'] == 'Winner':
        n_oscars+=1

  return n_oscars

def insert_num_oscars(imdb, movie_table):
  for index, row in movie_table.iterrows():
    movie_id = int(row['id_IMDB'][2:])
    num_oscars = get_num_oscars(imdb, movie_id)
    movie_table.loc[index,'num_oscars'] = int(num_oscars)

In [14]:
# Inserir número de Oscars dos filmes usando IMDB
imdb = IMDb()
movies_table = pd.read_csv('filme.csv')
insert_num_oscars(imdb, movies_table)
movies_table.to_csv('filme.csv', encoding='utf-8', index=False)

In [11]:
# movies_table = pd.read_csv('filme.csv')
# print(movies_table.columns)
# movies_table = movies_table.drop(columns=['nacionalidade'])
# print(movies_table.columns)
movies_table.to_csv('filme.csv', encoding='utf-8', index=False)

# **Testes**

In [ ]:
insert_movie_certification(movies_table)

In [ ]:
#imdb = IMDb()

#m1 = imdb.get_movie('9770150')
for i in m1.keys():
  print(i)

In [ ]:
#print(m1['awards'])

#m1 = imdb.get_movie('4154796', info=['quotes', 'goofs', 'reviews', 'release info'])
for i in m1['awards']:
  print(i)

In [35]:
# imdb.get_person_infoset()
# person = imdb.get_person('scarlett johansson')
#print(person)
person_awards = imdb.get_person('0424060', info=['awards'])
person_awards = imdb.get_person_awards('0424060')

for i in person_awards['data']['awards']:
  print(i)

{'year': 2020, 'result': 'Nominee', 'prize': 'Oscar', 'category': 'Best Performance by an Actress in a Leading Role', 'movies': <Movie id:7653254[http] title:_Marriage Story (2019)_>, 'award': 'Academy Awards, USA'}
{'category': 'Best Performance by an Actress in a Supporting Role', 'movies': <Movie id:2584384[http] title:_Jojo Rabbit (2019)_>, 'award': 'Academy Awards, USA'}
{'year': 2020, 'result': 'Nominee', 'prize': 'BAFTA Film Award', 'category': 'Best Supporting Actress', 'movies': <Movie id:2584384[http] title:_Jojo Rabbit (2019)_>, 'award': 'BAFTA Awards'}
{'category': 'Best Leading Actress', 'movies': <Movie id:7653254[http] title:_Marriage Story (2019)_>, 'award': 'BAFTA Awards'}
{'year': 2004, 'result': 'Winner', 'prize': 'BAFTA Film Award', 'category': 'Best Performance by an Actress in a Leading Role', 'movies': <Movie id:0335266[http] title:_Lost in Translation (2003)_>, 'award': 'BAFTA Awards'}
{'result': 'Nominee', 'prize': 'BAFTA Film Award', 'category': 'Best Performa

In [ ]:
#imdb.get_movie_infoset()
#m1.infoset2keys
m1 = imdb.get_movie('0167260', info=['awards', 'plot'])

In [ ]:
movies_table = pd.read_csv('filme.csv')
print(movies_table.head())
print(movies_table.columns)


   id_TMDB    id_IMDB  ...     receita num_oscars
0    19995  tt0499549  ...  2847246203    470.550
1   299534  tt4154796  ...  2797800564    227.636
2   140607  tt2488496  ...  2068223624     77.785
3   299536  tt4154756  ...  2046239637    311.316
4   135397  tt0369610  ...  1671713208     97.764

[5 rows x 14 columns]
Index(['id_TMDB', 'id_IMDB', 'titulo', 'titulo_original', 'sinopse', 'duracao',
       'ano', 'classificacao', 'situacao', 'nacionalidade', 'idioma_original',
       'orcamento', 'receita', 'num_oscars'],
      dtype='object')
